In [1]:
# Seed for reproducibility
import torch
import numpy as np
import pandas as pd
import scanpy as sc
from typing import Tuple

import sys
sys.path.append('..')
from tools.formating.formating import *
from tools.utils.utils import run_dimension_reduction
from tools.evaluation.monitor import *

# scVI imports
import scvi
print(scvi.__version__)
from scvi.model.utils import mde
import pymde

torch.manual_seed(0)
np.random.seed(0)
sc.settings.verbosity = 0  # verbosity: errors (0), warnings (1), info (2), hints (3)

/usr/local/lib/python3.8/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.8/dist-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda9SetDeviceEi'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
Global seed set to 0


0.20.3


In [2]:
input = "/ps/ai-ready/data/Clustering/Tabula_Sapiens/TS_Skin_test.h5ad"
adata = load_anndata(input)
adata

/ps/ai-ready/data/Clustering/Tabula_Sapiens/TS_Skin_test.h5ad


AnnData object with n_obs × n_vars = 725 × 3853
    obs: 'organ_tissue', 'method', 'donor', 'anatomical_information', 'n_counts_UMIs', 'n_genes', 'cell_ontology_class', 'free_annotation', 'manually_annotated', 'compartment', 'gender', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'outlier', 'mt_outlier', 'doublet_scores', 'predicted_doublets'
    var: 'gene_symbol', 'feature_type', 'ensemblid', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std', 'n_cells', 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
    uns: '_scvi', '_training_mode', 'cell_ontology_class_colors', 'dendrogram_cell_type_tissue', 'dendrogram_comput

In [3]:
monitor = Monitor(1)

In [4]:
# scVI uses non normalized data so we keep the original data in a separate AnnData object, then the normalization steps are performed
scvi.model.SCVI.setup_anndata(
    adata,
    layer="raw_counts"
)

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [5]:
model = scvi.model.SCVI(adata)
model

SCVI Model with the following params: 
n_hidden: 128, n_latent: 10, n_layers: 1, dropout_rate: 0.1, dispersion: gene, gene_likelihood: zinb, 
latent_distribution: normal
Training status: Not Trained
Model's adata is minified?: False

In [6]:
model.train()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/trainer.py:1609: PossibleUserWarning: The number of training batches (6) is smaller than the logging interval Trainer(log_every_n_steps=10). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 400/400: 100%|█████████████████████████████████████████████████████████| 400/400 [01:03<00:00,  6.35it/s, loss=1.28e+03, v_num=1]

`Trainer.fit` stopped: `max_epochs=400` reached.


Epoch 400/400: 100%|█████████████████████████████████████████████████████████| 400/400 [01:03<00:00,  6.31it/s, loss=1.28e+03, v_num=1]


In [7]:
latent = model.get_latent_representation()
adata.obsm["X_scVI"] = latent

In [8]:
denoised = model.get_normalized_expression(adata, library_size=1e4)
adata.layers["scvi_normalized"] = model.get_normalized_expression(
    library_size=10e4
)

In [9]:
adata.layers["raw_counts"].toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

In [10]:
adata.obsm["X_scVI"].shape[1]

10

In [12]:
adata = run_dimension_reduction(adata, use_rep='X_scVI', n_pcs=10, n_neighbors=15)

In [ ]:
adata.obsm["X_mde"] = mde(adata.obsm["X_scVI"])

In [ ]:
time_points, cpu_usage, mem_usage = monitor.stop()

In [ ]:
time_points

In [ ]:
cpu_usage

In [ ]:
mem_usage